# dataset download
https://grouplens.org/datasets/movielens/25m/

wget -i https://files.grouplens.org/datasets/movielens/ml-25m.zip -P /Users/auto/github/spark/data/ml-25m

In [1]:
from pyspark.sql import SparkSession

In [2]:
MAX_MEMORY = "20g"
spark = SparkSession.builder.appName("movie-recommendation")\
    .config("spark.executor.memory", MAX_MEMORY)\
    .config("spark.driver.memory", MAX_MEMORY)\
    .getOrCreate()

22/12/26 01:22:44 WARN Utils: Your hostname, PSui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.7 instead (on interface en0)
22/12/26 01:22:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/26 01:22:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/26 01:22:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/12/26 01:22:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/12/26 01:22:44 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/12/26 01:22:44 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


# out of memory 발생
22/12/26 01:16:05 ERROR Executor: Exception in task 1.0 in stage 9.0 (TID 45)
java.lang.OutOfMemoryError: Java heap space

In [3]:
# spark = SparkSession.builder.appName("movie-recommendation").getOrCreate()

In [4]:
ratings_file = '../../../../data/ml-25m/ratings.csv'
ratings_df = spark.read.csv(f"{ratings_file}", inferSchema=True, header=True)

In [5]:
ratings_df.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows



In [6]:
ratings_df = ratings_df.select(["userId", "movieId", "rating"])

In [7]:
ratings_df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [8]:
ratings_df.select(ratings_df.rating).describe().show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|          25000095|
|   mean| 3.533854451353085|
| stddev|1.0607439611423666|
|    min|               0.5|
|    max|               5.0|
+-------+------------------+



In [9]:
train_df, test_df = ratings_df.randomSplit([0.8, 0.2])

In [10]:
from pyspark.ml.recommendation import ALS
# maxIter 최고 반복 횟수
# regParam = default 1.0,
als = ALS(maxIter=5,
          regParam=0.1,    # regularization default 1.0
          userCol="userId",
          itemCol="movieId",
          ratingCol='rating',
          coldStartStrategy='drop'  # 학습하지 못한 데이터를 만났을 때 어떻게 대처할 것인지 개발자가 설정하는 것
          )

In [11]:
model = als.fit(train_df)

22/12/26 01:23:04 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/12/26 01:23:04 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/12/26 01:23:04 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
22/12/26 01:23:05 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [12]:
predictions = model.transform(test_df)

In [13]:
predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   101|   8638|   5.0|  3.563535|
|   243|   1580|   3.0| 2.6667275|
|   296|   1580|   3.0| 2.7252097|
|   322|   1645|   4.0| 3.6049063|
|   368|   3175|   5.0|    3.6444|
|   497|   2366|   4.0| 3.7677696|
|   513|  44022|   5.0|  4.288964|
|   596|   1580|   3.0| 3.5645356|
|   597|   1088|   3.0| 3.3457541|
|   597|   3997|   1.0| 1.9579154|
|   613|   1645|   4.0| 3.6142502|
|   756|   3175|   2.5| 3.2250512|
|   756|  44022|   3.5| 3.0737169|
|   830|   1580|   5.0| 3.5827847|
|   830|   1591|   2.0|   2.82837|
|   847|    471|   5.0| 3.4477873|
|   847|   1959|   2.0| 3.2866914|
|   847|   4818|   0.5| 1.2071182|
|   847|   6357|   3.5| 3.3180442|
|   847| 180981|   3.5| 3.3255274|
+------+-------+------+----------+
only showing top 20 rows



In [14]:
predictions.select('rating', 'prediction').describe().show()

+-------+-----------------+------------------+
|summary|           rating|        prediction|
+-------+-----------------+------------------+
|  count|          4996988|           4996988|
|   mean|3.534111048495614| 3.425122786307677|
| stddev|1.060547437977407|0.6463546760818288|
|    min|              0.5|        -2.1740968|
|    max|              5.0|          6.361507|
+-------+-----------------+------------------+



In [15]:
# evaluation
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName='rmse',
                                labelCol='rating',
                                predictionCol='prediction')

In [16]:
rmse = evaluator.evaluate(predictions)

In [17]:
print(rmse)

0.8086567324436121


### user마다 recommendation 추출

In [18]:
model.recommendForAllUsers(3).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    26|[{203086, 5.48622...|
|    27|[{203086, 5.91870...|
|    28|[{151989, 7.44217...|
|    31|[{201821, 3.89801...|
|    34|[{194434, 5.90089...|
|    53|[{203882, 6.49384...|
|    65|[{144202, 6.55153...|
|    76|[{194434, 6.05897...|
|    78|[{203882, 6.71608...|
|    81|[{189161, 4.49030...|
|    85|[{128667, 6.30716...|
|   101|[{203086, 5.15773...|
|   108|[{203086, 5.41597...|
|   115|[{203086, 6.55031...|
|   126|[{203086, 6.45011...|
|   133|[{151989, 5.33744...|
|   137|[{203086, 5.66314...|
|   148|[{203086, 5.69119...|
|   155|[{194434, 6.07723...|
|   183|[{151989, 6.18763...|
+------+--------------------+
only showing top 20 rows



In [19]:
model.recommendForAllItems(3).show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     26|[{105130, 5.02717...|
|     27|[{87426, 5.49956}...|
|     28|[{105801, 5.42771...|
|     31|[{87426, 5.360557...|
|     34|[{58248, 5.467169...|
|     53|[{20126, 5.196907...|
|     65|[{87426, 5.184136...|
|     76|[{87426, 5.262904...|
|     78|[{142811, 4.72128...|
|     81|[{156318, 4.78692...|
|     85|[{84907, 4.866537...|
|    101|[{56026, 4.933163...|
|    108|[{31506, 5.562128...|
|    115|[{105801, 5.62139...|
|    126|[{87426, 5.010902...|
|    133|[{80084, 4.378054...|
|    137|[{100653, 4.98371...|
|    148|[{142811, 4.21368...|
|    155|[{10417, 5.097225...|
|    183|[{10417, 4.993128...|
+-------+--------------------+
only showing top 20 rows



# 특정 유저에 대해서 recommendation 생성

In [20]:
from pyspark.sql.types import IntegerType

user_list = [133, 81, 101]

users_df = spark.createDataFrame(user_list, IntegerType()).toDF('userId')
users_df.show()

+------+
|userId|
+------+
|   133|
|    81|
|   101|
+------+



In [27]:
user_recs = model.recommendForUserSubset(users_df, 5)
user_recs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   133|[{151989, 5.33744...|
|   101|[{203086, 5.15773...|
|    81|[{189161, 4.49030...|
+------+--------------------+



In [28]:
movies_list = user_recs.collect()[0].recommendations

In [32]:
recs_df = spark.createDataFrame(movies_list)
recs_df.show()

+-------+------------------+
|movieId|            rating|
+-------+------------------+
| 151989| 5.337449550628662|
| 201821|5.2547502517700195|
| 203882| 5.252063274383545|
| 183947| 5.246218204498291|
| 194434| 5.164909839630127|
+-------+------------------+



# file 이름 가져오기


In [33]:
movies_file = '../../../../data/ml-25m/movies.csv'
movies_df = spark.read.csv(f"{movies_file}", inferSchema=True, header=True)

In [34]:
movies_df.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [35]:
recs_df.createOrReplaceTempView("recommendations")
movies_df.createOrReplaceTempView('movies')

In [38]:
query = """
SELECT *
FROM
    movies JOIN recommendations ON recommendations.movieId = movies.movieId
ORDER BY
    rating DESC
"""
recommended_movies = spark.sql(query)
recommended_movies.show()

+-------+--------------------+------------------+-------+------------------+
|movieId|               title|            genres|movieId|            rating|
+-------+--------------------+------------------+-------+------------------+
| 151989|    The Thorn (1971)|            Comedy| 151989| 5.337449550628662|
| 201821| Civilisation (1969)|(no genres listed)| 201821|5.2547502517700195|
| 203882|Dead in the Water...|            Horror| 203882| 5.252063274383545|
| 183947|NOFX Backstage Pa...|(no genres listed)| 183947| 5.246218204498291|
| 194434|   Adrenaline (1990)|(no genres listed)| 194434| 5.164909839630127|
+-------+--------------------+------------------+-------+------------------+



# API 형태 delivery 가능한 함수

In [46]:
def get_recommendations(user_id, num_recs):
    inner_query = """
        SELECT *
        FROM
            movies JOIN recommendations ON recommendations.movieId = movies.movieId
        ORDER BY
            rating DESC
    """

    inner_users_df = spark.createDataFrame([user_id], IntegerType()).toDF("userId")
    inner_user_recs_df = model.recommendForUserSubset(inner_users_df, num_recs)

    recs_list = inner_user_recs_df.collect()[0].recommendations
    inner_recs_df = spark.createDataFrame(recs_list)

    inner_recs_df.createOrReplaceTempView("recommendations")
    movies_df.createOrReplaceTempView('movies')

    inner_recommended_movies = spark.sql(inner_query)
    return inner_recommended_movies


In [47]:
# 456 아이디의 유저, 10개의 추천을 해라.
recs = get_recommendations(456, 10)

In [48]:
recs.toPandas()

,movieId,title,genres,movieId,rating
0,203086,Truth and Justice (2019),Drama,203086,6.712993
1,203882,Dead in the Water (2006),Horror,203882,6.265034
2,194434,Adrenaline (1990),(no genres listed),194434,6.242795
3,157791,.hack Liminality In the Case of Kyoko Tohno,(no genres listed),157791,6.235188
4,157789,.hack Liminality In the Case of Yuki Aihara,(no genres listed),157789,6.235188
5,166812,Seeing Red: Stories of American Communists (1983),(no genres listed),166812,6.193880
6,151989,The Thorn (1971),Comedy,151989,6.083502
7,184723,Little Boy Blue (2017),Crime|Drama,184723,6.077363
8,120821,The War at Home (1979),Documentary|War,120821,6.062615
9,206943,Cássia (2015),Documentary,206943,6.055067


In [49]:
spark.stop()